In [1]:
import torch
import torch.nn as nn
torch.set_default_dtype(torch.float64)
from collections import OrderedDict
import math
import numpy as np
import sympy as sp
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.stats import qmc
import scipy.io

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
class DNN1(torch.nn.Module):
    def __init__(self, layers):
        super(DNN1, self).__init__()
        
        self.depth = len(layers) - 1
        
        # 设置激活函数
        self.activation1 = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation1()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        
        # # 使用Lambda层实现 -exp(x)
        # layer_list.append(('output_activation', LambdaLayer(lambda x: -torch.exp(x))))
        
        layerDict = OrderedDict(layer_list)
        
        # 部署层
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, X):
        out = self.layers(X)
        return out
    
    def weights_init(m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0.0)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
        
    def forward(self, x):
        return self.lambd(x)


In [4]:
# the physics-guided neural network
class PINN_theta(torch.nn.Module):
    def __init__(self, X_ub,X_lb,X_ic,X_res,u_ic,v_ic,alpha,beta,gamma,layers):
        super(PINN_theta, self).__init__()
        
        # data
        self.z_ub = torch.tensor(X_ub[:, 0:1], requires_grad=True).double().to(device)
        self.t_ub = torch.tensor(X_ub[:, 1:2], requires_grad=True).double().to(device)

        self.z_lb = torch.tensor(X_lb[:, 0:1], requires_grad=True).double().to(device)
        self.t_lb = torch.tensor(X_lb[:, 1:2], requires_grad=True).double().to(device)

        self.z_ic = torch.tensor(X_ic[:, 0:1], requires_grad=True).double().to(device)
        self.t_ic = torch.tensor(X_ic[:, 1:2], requires_grad=True).double().to(device)
        self.u_ic = torch.tensor(u_ic).double().to(device)
        self.v_ic = torch.tensor(v_ic).double().to(device)

        self.z_res = torch.tensor(X_res[:, 0:1], requires_grad=True).double().to(device)
        self.t_res = torch.tensor(X_res[:, 1:2], requires_grad=True).double().to(device)

        self.alpha = torch.tensor(alpha).double().to(device)
        self.beta = torch.tensor(beta).double().to(device)
        self.gamma = torch.tensor(gamma).double().to(device)


        self.layers_theta = layers
        self.dnn_theta = DNN1(layers).to(device)

        self.iter = 0
        #self.pre_iter = 0

        self.optimizer1 = torch.optim.Adam(self.dnn_theta.parameters(),lr=1e-3)

        self.optimizer2 = torch.optim.LBFGS(
            self.dnn_theta.parameters(), 
            lr=1.0, #步长缩放因子，用于控制拟牛顿算法中搜索的步长
            max_iter=40000, #最大迭代次数
            max_eval=40000, #函数和梯度最大评估次数
            history_size=50, #存储和更新历史信息的缓存大小
            tolerance_grad=1e-7, #表示梯度的容差，即梯度的变化小于该容差时，算法将终止
            tolerance_change=1.0 * np.finfo(float).eps, #表示优化变化的容差，即参数变化量的绝对值小于该容差时，算法将终止
            line_search_fn="strong_wolfe" #can be "strong_wolfe"
        )

        self.loss_history = {
            'total': [],
            'residual': [],
            'ic': [],
            'b': []
        }

    def net_h(self, z, t):  
        H = self.dnn_theta(torch.cat([z, t], dim=1))
        u = H[:, 0:1]
        v = H[:, 1:2]
        return u,v
        
    def gradient(self, y, x):
        return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), retain_graph=True, create_graph=True)[0]
    
    def net_b(self, z, t):

        u,v = self.net_h(z, t)
        u_z = self.gradient(u, z)
        v_z = self.gradient(v, z)

        return u, v, u_z, v_z            
    
    def net_ic(self, z, t):

        u, v = self.net_h(z, t)

        return u, v      
    
    def net_f(self, z, t):

        u,v = self.net_h(z, t)
        v_t = self.gradient(v, t)
        v_z = self.gradient(v, z)

        u_t = self.gradient(u, t)
        u_z = self.gradient(u, z)
        u_zz = self.gradient(u_z, z)
        u_zzz = self.gradient(u_zz, z)

        f1 = u_t - v_z
        f2 = v_t + self.alpha * u_z + self.beta * (2*u*u_z) + self.gamma * u_zzz

        return f1, f2

    def loss_func(self):

        self.optimizer1.zero_grad()
        self.optimizer2.zero_grad()

        u_ic_pred, v_ic_pred = self.net_ic(self.z_ic, self.t_ic)

        u_ub_pred, v_ub_pred, u_z_ub_pred, v_z_ub_pred = self.net_b(self.z_ub, self.t_ub)

        u_lb_pred, v_lb_pred, u_z_lb_pred, v_z_lb_pred = self.net_b(self.z_lb, self.t_lb)

        f1_res_pred, f2_res_pred = self.net_f(self.z_res, self.t_res)

        loss_res1 = torch.mean(f1_res_pred**2)
        loss_res2 = torch.mean(f2_res_pred**2)
        loss_res = loss_res1 + loss_res2
        
        loss_ic1 = torch.mean((u_ic_pred - self.u_ic)**2)
        loss_ic2 = torch.mean((v_ic_pred - self.v_ic)**2)
        loss_b1 = torch.mean((u_ub_pred - u_lb_pred)**2)
        loss_b2 = torch.mean((v_ub_pred - v_lb_pred)**2)
        loss_b3 = torch.mean((u_z_ub_pred - u_z_lb_pred)**2)
        loss_b4 = torch.mean((v_z_ub_pred - v_z_lb_pred)**2)

        loss_ic = loss_ic1 + loss_ic2
        loss_b = loss_b1 + loss_b2 + loss_b3 + loss_b4

        loss = loss_res + loss_ic1 + loss_ic2 + loss_b1 + loss_b2 + loss_b3 + loss_b4
        
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            self.loss_history['total'].append(loss.item())
            self.loss_history['residual'].append(loss_res.item())
            self.loss_history['ic'].append(loss_ic.item())
            self.loss_history['b'].append(loss_b.item())   
            print(
                'Iter %d, Loss: %.5e, Loss_res: %.5e, Loss_ic1: %.5e, Loss_ic2: %.5e, Loss_b1: %.5e, Loss_b2: %.5e, Loss_b3: %.5e, Loss_b4: %.5e' % 
                (   self.iter, 
                    loss.item(), 
                    loss_res.item(), 
                    loss_ic1.item(),
                    loss_ic2.item(), 
                    loss_b1.item(),
                    loss_b2.item(),
                    loss_b3.item(),
                    loss_b4.item()
                )
            )

        return loss
    
    def train(self):
        self.dnn_theta.train()
        print("采用Adam优化器")
        for i in range(20000):
            self.optimizer1.step(self.loss_func)
        # 然后运行lbfgs优化器
        print("采用L-BFGS优化器")
        self.optimizer2.step(self.loss_func) 
            
    def predict(self, X):
        
        z = torch.tensor(X[:, 0:1], requires_grad=True).double().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).double().to(device)

        self.dnn_theta.eval()
        u,v = self.net_h(z, t)
        u = u.detach().cpu().numpy()
        v = v.detach().cpu().numpy()

        return u, v

In [5]:
# alpha = -1
# beta = -3
# gamma = -1

# N_res = 1000
# N_b = 100
# N_ic = 100

# x = np.linspace(-15, 15, 201)
# t = np.linspace(0, 5, 201) 
# x_mesh, t_mesh = np.meshgrid(x, t)

# X = np.hstack((x_mesh.flatten()[:,None], t_mesh.flatten()[:,None]))

# lb = X.min(0) #下界x'QxQ
# ub = X.max(0)  #上界 

# def lhs(n, size):
#     """生成n维拉丁超立方采样，返回size个样本点"""
#     sampler = qmc.LatinHypercube(d=n)
#     return sampler.random(n=size)

# # 使用简洁形式进行采样（与你要求的形式一致）
# X_res = lb + (ub - lb) * lhs(2, N_res)  # 2维残差点采样

# t_samples = lb[1] + (ub[1] - lb[1]) * lhs(1, N_b)  # 时间t的采样：[N_b, 1]

# # 上边界点（x=ub[0]，与下边界对称）
# X_ub = np.hstack((ub[0] * np.ones((N_b, 1)),t_samples))
# X_lb = np.hstack((lb[0] * np.ones((N_b, 1)),t_samples))

# X_ic = np.hstack((lb[0] + (ub[0] - lb[0]) * lhs(1, N_ic), 
#                   lb[1] * np.ones((N_ic, 1))))  # 初始条件点

In [6]:
alpha = -1
beta = -3
gamma = -1

x = np.linspace(-15, 15, 301)
t = np.linspace(0, 5, 201) 
x_mesh, t_mesh = np.meshgrid(x, t)

X = np.hstack((x_mesh.flatten()[:,None], t_mesh.flatten()[:,None]))

lb = X.min(0) #下界x'QxQ
ub = X.max(0)  #上界 

df_h = np.load("/kaggle/input/bq-soliton-residual-0-15-10000points/residual_10000.npz")
X_res = df_h['X_res']
X_ub = df_h['X_ub']
X_lb = df_h['X_lb']
X_ic = df_h['X_ic']

In [7]:
def func_u(X):
    x, t = X[:, 0:1], X[:, 1:2]
    zeta = 1/6
    y1 = 0
    y2 = 0
    y3 = 2
    k1 = 1
    k2 = 1
    k3 = 5/4
    varsigma1 = -2
    varsigma2 = 4
    varsigma3 = -4

    u=-alpha/(2*beta) + zeta + 6*gamma*(-k1**2*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - k2**2*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - k3**2*np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) + (k1 + k2)**2*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (k1 + k3)**2*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k2 + k3)**2*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k1 + k2 + k3)**2*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)) - (-k1*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - k2*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - k3*np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) + (k1 + k2)*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (k1 + k3)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k2 + k3)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k1 + k2 + k3)*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)))**2/((2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)) + (2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) - np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) - 1))/(beta*((2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)) + (2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) - np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) - 1))
    return u

def func_v(X):
    x, t = X[:, 0:1], X[:, 1:2]
    zeta = 1/6
    y1 = 0
    y2 = 0
    y3 = 2
    k1 = 1
    k2 = 1
    k3 = 5/4
    varsigma1 = -2
    varsigma2 = 4
    varsigma3 = -4

    v=6*gamma*(-k1**2*np.sqrt(-2*beta*zeta - gamma*k1**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) + k2**2*np.sqrt(-2*beta*zeta - gamma*k2**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - k3**2*np.sqrt(-2*beta*zeta - gamma*k3**2)*np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) + (k1 + k2)*(k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (k1 + k3)*(k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) - (k2 + k3)*(k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k1 + k2 + k3)*(k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)) - (-k1*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - k2*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - k3*np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) + (k1 + k2)*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (k1 + k3)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k2 + k3)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k1 + k2 + k3)*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)))*(-k1*np.sqrt(-2*beta*zeta - gamma*k1**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2)*np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) - (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))*(2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)))/((2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)) + (2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) - np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) - 1))/(beta*((2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma2 + varsigma3)/((2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)) + (2*beta*zeta*(k1 - k2)**2 + gamma*(k1 - k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) - k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma1 + varsigma2)/(2*beta*zeta*(k1 + k2)**2 + gamma*(k1 + k2)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k2*np.sqrt(-2*beta*zeta - gamma*k2**2))**2) + (2*beta*zeta*(k1 - k3)**2 + gamma*(k1 - k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma1 + varsigma3)/(2*beta*zeta*(k1 + k3)**2 + gamma*(k1 + k3)**4 + (k1*np.sqrt(-2*beta*zeta - gamma*k1**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) + (2*beta*zeta*(k2 - k3)**2 + gamma*(k2 - k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) + k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2)*np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma2 + varsigma3)/(2*beta*zeta*(k2 + k3)**2 + gamma*(k2 + k3)**4 + (k2*np.sqrt(-2*beta*zeta - gamma*k2**2) - k3*np.sqrt(-2*beta*zeta - gamma*k3**2))**2) - np.exp(k1*t*np.sqrt(-2*beta*zeta - gamma*k1**2) + k1*(x + y1) + varsigma1) - np.exp(-k2*t*np.sqrt(-2*beta*zeta - gamma*k2**2) + k2*(x + y2) + varsigma2) - np.exp(k3*t*np.sqrt(-2*beta*zeta - gamma*k3**2) + k3*(x + y3) + varsigma3) - 1))
    return v

u_ic = func_u(X_ic)
v_ic = func_v(X_ic)


In [8]:
layers = [2] + 6 * [80] + [2]
model = PINN_theta(X_ub,X_lb,X_ic,X_res,u_ic,v_ic,alpha,beta,gamma,layers)

In [9]:
def count_parameters(model):
    """计算模型中可训练参数的总个数"""
    total_params = 0
    for param in model.parameters():
        # 累加每个参数张量的元素数量
        total_params += param.numel()
    return total_params

param_count = count_parameters(model)
print(f"模型可训练参数总个数: {param_count:,}")  # 使用逗号作为千位分隔符

模型可训练参数总个数: 32,802


In [10]:
model.train()

采用Adam优化器
Iter 100, Loss: 3.93984e-02, Loss_res: 4.38846e-03, Loss_ic1: 1.22425e-02, Loss_ic2: 2.11031e-02, Loss_b1: 9.52018e-05, Loss_b2: 1.16820e-03, Loss_b3: 3.81194e-04, Loss_b4: 1.96936e-05
Iter 200, Loss: 3.45828e-02, Loss_res: 2.93125e-03, Loss_ic1: 9.01977e-03, Loss_ic2: 1.90593e-02, Loss_b1: 3.07789e-03, Loss_b2: 5.84483e-05, Loss_b3: 3.43443e-04, Loss_b4: 9.26275e-05
Iter 300, Loss: 2.77632e-02, Loss_res: 2.43539e-03, Loss_ic1: 7.46458e-03, Loss_ic2: 1.71444e-02, Loss_b1: 3.18018e-05, Loss_b2: 2.52435e-04, Loss_b3: 2.57768e-04, Loss_b4: 1.76763e-04
Iter 400, Loss: 2.63239e-02, Loss_res: 2.19903e-03, Loss_ic1: 7.03323e-03, Loss_ic2: 1.60119e-02, Loss_b1: 1.65106e-04, Loss_b2: 4.83670e-04, Loss_b3: 2.25175e-04, Loss_b4: 2.05796e-04
Iter 500, Loss: 2.37844e-02, Loss_res: 2.24312e-03, Loss_ic1: 6.51303e-03, Loss_ic2: 1.43982e-02, Loss_b1: 2.53778e-05, Loss_b2: 8.52483e-05, Loss_b3: 2.14062e-04, Loss_b4: 3.05362e-04
Iter 600, Loss: 2.22743e-02, Loss_res: 2.18318e-03, Loss_ic1: 6.2

In [11]:
all_states1 = {"model": model.dnn_theta.state_dict(), "LBFGS": model.optimizer2.state_dict()}
torch.save(obj=all_states1, f="/kaggle/working/all_states1.pth")

In [12]:
u_p,v_p = model.predict(X)
u_true = func_u(X)
v_true = func_v(X)

In [13]:
L2 = np.linalg.norm(u_p.flatten() - u_true.flatten())/np.linalg.norm(u_true.flatten())

print (L2)

0.0007678478350266826


In [14]:
L2 = np.linalg.norm(v_p.flatten() - v_true.flatten())/np.linalg.norm(v_true.flatten())

print (L2)

0.0007022036946653833


In [15]:
pinndata = {
    'x': x_mesh.flatten()[:, None],       # x网格数据（竖排）
    't': t_mesh.flatten()[:, None],       # t网格数据（竖排）
    'h_true': u_true.flatten()[:, None],  # u真实值（竖排）
    'g_true': v_true.flatten()[:, None],  # v真实值（竖排）
    'h_pred': u_p.flatten()[:, None],     # u预测值（竖排）
    'g_pred': v_p.flatten()[:, None]      # v预测值（竖排）
}

# 保存到单个MAT文件
scipy.io.savemat('pinnall_data.mat', pinndata)

In [16]:
loss_total = model.loss_history['total']
loss_residual = model.loss_history['residual']
loss_ic = model.loss_history['ic']
loss_b = model.loss_history['b']

In [17]:
num_iterations = len(loss_total)  # 损失记录的总数量（即总迭代次数）
iter_steps = list(range(100, 100 + num_iterations * 100, 100))

In [18]:
# 3. 将数据转换为竖排（列向量）：通过reshape(-1, 1)实现，形状变为(n, 1)
iter_steps_col = np.array(iter_steps).reshape(-1, 1)       # 竖排迭代步数
loss_total_col = np.array(loss_total).reshape(-1, 1)       # 竖排总损失
loss_residual_col = np.array(loss_residual).reshape(-1, 1) # 竖排残差损失
loss_ic_col = np.array(loss_ic).reshape(-1, 1)             # 竖排初始条件损失
loss_b_col = np.array(loss_b).reshape(-1, 1)               # 竖排边界条件损失

# 4. 整理并保存为mat文件（此时所有数据均为列向量）
data_loss = {
    'iter_steps': iter_steps_col,
    'loss_total': loss_total_col,
    'loss_residual': loss_residual_col,
    'loss_ic': loss_ic_col,
    'loss_b': loss_b_col
}

scipy.io.savemat('pinnloss_history.mat', data_loss)

In [19]:
# scipy.io.savemat('3wavedata_PINN/x.mat', {'array': x_mesh.flatten()[:, None]})
# scipy.io.savemat('3wavedata_PINN/t.mat', {'array': t_mesh.flatten()[:, None]})
# scipy.io.savemat('3wavedata_PINN/h_true.mat', {'array': u_true.flatten()[:, None]})
# scipy.io.savemat('3wavedata_PINN/g_true.mat', {'array': v_true.flatten()[:, None]})
# scipy.io.savemat('3wavedata_PINN/h_pred.mat', {'array': u_p.flatten()[:, None]})
# scipy.io.savemat('3wavedata_PINN/g_pred.mat', {'array': v_p.flatten()[:, None]})